In [1]:
import pandas as pd

,rop_rxnstr,rop_rxncomment,rop_spcname,rop
0,"1,3-BUTADIENE(L)+CD<=>CC=CC1CC=CCC1C","1,3-BUTADIENE(L) + CD Diels-Alder addition",CD,-5.434319e-17
1,"1,3-BUTADIENE(L)+CD<=>C=CC1CC(C)C=CC1C","1,3-BUTADIENE(L) + CD Diels-Alder addition",CD,-8.793038e-16
2,"1,3-BUTADIENE(L)+CD<=>C=CC1CC(C)C=CC1C","1,3-BUTADIENE(L) + CD Diels-Alder addition",CD,-8.793038e-16
3,CYCLOPENTADIENE(L)+CD<=>CC=CC1C2C=CC(C2)C1C,CYCLOPENTADIENE(L) + CD Diels-Alder addition,CD,-7.188601e-18
4,CYCLOPENTADIENE(L)+CD<=>CC1C=CC(C)C2CC=CC12,CYCLOPENTADIENE(L) + CD Diels-Alder addition,CD,-7.188601e-18
...,...,...,...,...
139291,"KR+COOH(L,oligomer)<=>CC(C)C+COO.(L,oligomer)","KR + COOH(L,oligomer) -> inert(S) + COO.(L,oli...",mass,1.495254e-24
139292,"PR+allylic_CH(L,oligomer)<=>HP+allylic_C.(L,ol...","PR + allylic_CH(L,oligomer) <=> HP + allylic_C...",mass,-2.618823e-20
139293,"PR+COOH(L,oligomer)<=>HP+COO.(L,oligomer)","PR + COOH(L,oligomer) <=> HP + COO.(L,oligomer)",mass,-2.783281e-23
139294,"OR+allylic_CH(L,oligomer)<=>OH+allylic_C.(L,ol...","OR + allylic_CH(L,oligomer) <=> OH + allylic_C...",mass,2.844905e-16


In [121]:
def get_film_rops_1D(
    film_rate_of_productions,
    tray,
    cell_inds,
    rop_names,
    loss_only=False,
    production_only=False,
    N=3,
):
    if isinstance(rop_names, str):
        rop_names = [rop_names]

    cell_ind = cell_inds[0]
    df = film_rate_of_productions[tray, cell_ind]
    df.set_index(["rop_rxnstr", "rop_spcname"], inplace=True, drop=False)

    name_inds = df["rop_spcname"].isin(rop_names)
    rop_rxncomments = df.loc[name_inds, "rop_rxncomment"]
    rop_rxnstrs = df.loc[name_inds, "rop_rxnstr"]
    rops = df.loc[name_inds, "rop"]

    for cell_ind in cell_inds[1:]:
        df = film_rate_of_productions[tray, cell_ind]
        df.set_index(["rop_rxnstr", "rop_spcname"], inplace=True, drop=False)
        rops += df.loc[name_inds, "rop"]

    if loss_only:
        inds = rops < 0
    elif production_only:
        inds = rops > 0
    else:
        inds = rops != 0

    rops = rops[inds]
    rop_rxncomments = rop_rxncomments[inds]
    rop_rxnstrs = rop_rxnstrs[inds]

    sorted_inds = rops.abs().argsort()[::-1]
    if len(sorted_inds) > N:
        sorted_inds = sorted_inds[:N]

    return (
        rops.iloc[sorted_inds],
        rop_rxncomments.iloc[sorted_inds],
        rop_rxnstrs.iloc[sorted_inds],
    )



In [123]:
num_cells = 3
tray = 40
cell_inds = range(1, num_cells + 1)
rop_names = ["mass"]
film_rate_of_productions = {}

for cell_ind in cell_inds:
    path = f"/home/gridsan/hwpang/Jobs/PolymerFoulingModeling/TraceO2Perturbed/trace_oxygen_perturbed_20230718/simulation_results/OXYGEN_1e0_3600.0_32.0/simulation_film_1D_rop_{tray}_3cells_cell{cell_ind}_callback_t0.csv"
    df = pd.read_csv(path)
    df.drop_duplicates(subset=["rop_rxnstr", "rop_spcname"], keep="first", inplace=True)
    film_rate_of_productions[tray, cell_ind] = df

get_film_rops_1D(
    film_rate_of_productions,
    tray,
    cell_inds,
    rop_names,
    loss_only=False,
    production_only=False,
    N=3,)

(rop_rxnstr                                   rop_spcname
 CYCLOPENTADIENE(L)+CDB<=>CCC1C2C=CC(C2)C1CC  mass           2.881108e-08
 AR+CYCLOPENTADIENE(L)<=>C=C(C)CC1C=C[CH]C1   mass           2.909237e-09
 1,3-BUTADIENE(L)+AR<=>C=C[CH]CCC(=C)C        mass           2.685184e-09
 Name: rop, dtype: float64,
 rop_rxnstr                                   rop_spcname
 CYCLOPENTADIENE(L)+CDB<=>CCC1C2C=CC(C2)C1CC  mass           CYCLOPENTADIENE(L) + CDB Diels-Alder addition
 AR+CYCLOPENTADIENE(L)<=>C=C(C)CC1C=C[CH]C1   mass                AR + CYCLOPENTADIENE(L) radical addition
 1,3-BUTADIENE(L)+AR<=>C=C[CH]CCC(=C)C        mass                  1,3-BUTADIENE(L) + AR radical addition
 Name: rop_rxncomment, dtype: object,
 rop_rxnstr                                   rop_spcname
 CYCLOPENTADIENE(L)+CDB<=>CCC1C2C=CC(C2)C1CC  mass           CYCLOPENTADIENE(L)+CDB<=>CCC1C2C=CC(C2)C1CC
 AR+CYCLOPENTADIENE(L)<=>C=C(C)CC1C=C[CH]C1   mass            AR+CYCLOPENTADIENE(L)<=>C=C(C)CC1C=C[CH]C1
 1,3-BU

In [120]:
tray = 40
cell_ind = 1
path = f"/home/gridsan/hwpang/Jobs/PolymerFoulingModeling/TraceO2Perturbed/trace_oxygen_perturbed_20230718/simulation_results/OXYGEN_1e0_3600.0_32.0/simulation_film_1D_rop_{tray}_3cells_cell{cell_ind}_callback_t0.csv"
df = pd.read_csv(path)
df.drop_duplicates(subset=["rop_rxnstr", "rop_spcname", "rop"], keep="first", inplace=True)
df
# df.set_index(["rop_rxnstr", "rop_spcname"], inplace=True, drop=False)
# df.loc[index, :]

,rop_rxnstr,rop_rxncomment,rop_spcname,rop
0,"1,3-BUTADIENE(L)+CD<=>CC=CC1CC=CCC1C","1,3-BUTADIENE(L) + CD Diels-Alder addition",CD,-5.434319e-17
1,"1,3-BUTADIENE(L)+CD<=>C=CC1CC(C)C=CC1C","1,3-BUTADIENE(L) + CD Diels-Alder addition",CD,-8.793038e-16
3,CYCLOPENTADIENE(L)+CD<=>CC=CC1C2C=CC(C2)C1C,CYCLOPENTADIENE(L) + CD Diels-Alder addition,CD,-7.188601e-18
4,CYCLOPENTADIENE(L)+CD<=>CC1C=CC(C)C2CC=CC12,CYCLOPENTADIENE(L) + CD Diels-Alder addition,CD,-7.188601e-18
5,CYCLOPENTADIENE(L)+CD<=>CC1C=CC(C)C2CC=CC12,CYCLOPENTADIENE(L) + CD Diels-Alder addition,CD,-1.231145e-17
...,...,...,...,...
139295,"KR+COOH(L,oligomer)<=>CC(C)C+COO.(L,oligomer)","KR + COOH(L,oligomer) -> inert(S) + COO.(L,oli...",mass,1.495254e-24
139296,"PR+allylic_CH(L,oligomer)<=>HP+allylic_C.(L,ol...","PR + allylic_CH(L,oligomer) <=> HP + allylic_C...",mass,-2.618823e-20
139297,"PR+COOH(L,oligomer)<=>HP+COO.(L,oligomer)","PR + COOH(L,oligomer) <=> HP + COO.(L,oligomer)",mass,-2.783281e-23
139298,"OR+allylic_CH(L,oligomer)<=>OH+allylic_C.(L,ol...","OR + allylic_CH(L,oligomer) <=> OH + allylic_C...",mass,2.844905e-16
